# Intialize

In [1]:
!pip install --no-index tensorflow-gpu numpy

Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/tensorflow_gpu-2.1.0-cp37-cp37m-linux_x86_64.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/numpy-1.18.2-cp37-cp37m-linux_x86_64.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/Keras_Preprocessing-1.1.0-py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/grpcio-1.25.0-cp37-cp37m-linux_x86_64.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/termcolor-1.1.0-py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/absl_py-0.7.1-py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/Keras_Applications-1.0.8-py3-none-any

In [2]:
!pip install --no-index /home/guest139/s3transfer/botocore
!pip install --no-index /home/guest139/s3transfer
!pip install --no-index /home/guest139/sacremoses
!pip install --no-index /home/guest139/transformers

Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /home/guest139/s3transfer/botocore
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/jmespath-0.9.4-py2.py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/docutils-0.15.2-py3-none-any.whl
  Created wheel for botocore: filename=botocore-1.15.37-py2.py3-none-any.whl size=6074726 sha256=f6586f3520f336e9bacbb123f094d249d4ae921d2a38e170e06b079b139f0263
  Stored in directory: /tmp/pip-ephem-wheel-cache-lcp7ippk/wheels/e9/ee/87/770ad509ddd67e7e26c3442d1787625e278440f1a11c95d7ec
Successfully built botocore
Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gener

## Load all files

In [21]:
import tensorflow as tf

en_file1 = '../data/train.lang1.no-punctuation/train.lang1'
en_file2 = '../data/train.en.no-punctuation/unaligned.en'
fr_file1 = '../data/train.lang2.no-punctuation/train.lang2'
fr_file2 = '../data/train.fr.no-punctuation/unaligned.fr'
# TODO: exclude the test set

# lang_files = [en_file1, en_file2, fr_file1, fr_file2] TODO
lang_files = [en_file2]

# Tokenize

## Tokenize all files (whole strings (TODO: substrings, capitalization))

In [22]:
from collections import Counter

# Add control tokens
my_counter = Counter()
for i in range(2):
    my_counter.update(["<START>", "<STOP>", "<UNK>", "<MASK>", "<SEP>", "<PAD>"])

In [23]:
def string_to_tokens(raw_string):
    return raw_string.split()

line_lengths = []
for lang_file in lang_files:
    with open(lang_file) as f:
        for line in f:
            tokens = string_to_tokens(line)
            line_lengths += [len(tokens)]
            for token in tokens:
                my_counter.update([token])    

## Keep tokens that occur more than once

In [24]:
import numpy as np
tokens = np.array(list(my_counter.keys()))
frequencies = np.array(list(my_counter.values()))

In [25]:
retained_tokens = tokens[frequencies > 1]

In [26]:
print(len(tokens))
print(len(retained_tokens))

60014
40145


## Create lookup table dict: string -> token

In [27]:
indices = np.array(range(len(retained_tokens)))

In [28]:
indices

array([    0,     1,     2, ..., 40142, 40143, 40144])

In [29]:
my_tokenizer_lut = dict(zip(retained_tokens,indices))

## Define functions for tokenizing, padding

In [30]:
def pad_tokens(token_list, max_length):
    if len(token_list) >= max_length:
        token_list = token_list[:max_length]
        token_list[(max_length-1)] = my_tokenizer_lut["<STOP>"]
    else:
        while len(token_list) < max_length:
            token_list = token_list + [my_tokenizer_lut["<PAD>"]]
    return token_list

SENTENCE_LENGTH = 32
def tokenize_string(raw_string, max_length=SENTENCE_LENGTH): # TODO: Better definition of sentence length
    token_list = [my_tokenizer_lut["<START>"]]
    
    for token in string_to_tokens(raw_string):
        if token in my_tokenizer_lut:
            token_list += [my_tokenizer_lut[token]]
        else:
            token_list += [my_tokenizer_lut["<UNK>"]]
    
    token_list += [my_tokenizer_lut["<STOP>"]]
    
    token_list = pad_tokens(token_list, max_length)
    
    return np.array(token_list)[None,:]
        
tokenize_string("The quick brown fox jumped over the lazy dog.")

array([[    0,     2,   292,  3228,  6307,  2514,   995,     7, 12783,
            2,     1,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,     5]])

In [31]:
def tokenize_file(filename):
    with open(filename) as f:
        tokens = []
        for idx, line in enumerate(f):
            if idx < 2:
                print(line)
            tokens += [tokenize_string(line)]
    return np.concatenate(tokens,axis=0)

x_true = tokenize_file(en_file2)
x_true_val = tokenize_file(en_file1)
x_true.shape

for the second phase of the trials we just had different sizes small medium large and extra - large it 's true

geng had been my host the previous january when i was the first us defense secretary to visit china acting as an interlocutor for president jimmy carter ’s administration

so too does the idea that accommodating religious differences is dangerous

mr president ladies and gentlemen the financial perspective outlines the scope of the eu ’s activities over coming years as well as providing a framework for such activities and determining how effective they will be



(474000, 32)

# Define masking strategy

In [32]:
def mask_tokens(true_tokens):
    non_pad_tokens = true_tokens != my_tokenizer_lut["<PAD>"]
    random_masking_seed = np.random.uniform(0,1,(SENTENCE_LENGTH,)) * non_pad_tokens
    
    masking_targets = 0.85 < random_masking_seed # 15%
    mask_token_targets = np.logical_and(0.85 < random_masking_seed, random_masking_seed < 0.85 + 0.15*0.8) # 80% of 15%
    random_token_targets = np.logical_and(1.0 - 0.1*0.15 < random_masking_seed, random_masking_seed < 1.0) # 10% of 15%
    
    masked_tokens = true_tokens.copy()
    masked_tokens[mask_token_targets] = my_tokenizer_lut["<MASK>"]
    masked_tokens[random_token_targets] = np.random.randint(0,len(my_tokenizer_lut),(random_token_targets.sum(),))
#     masked_tokens[~mask_token_targets] = my_tokenizer_lut["<PAD>"]

    masked_true_tokens = true_tokens.copy()
    masked_true_tokens[~masking_targets] = my_tokenizer_lut["<PAD>"]
    
    attention_mask = true_tokens != my_tokenizer_lut["<PAD>"]
    
    return masked_tokens, masking_targets, masked_true_tokens, attention_mask

x_train, targets_train, masked_x_true, attention_mask = mask_tokens(x_true)
x_train_val, targets_train_val, masked_x_true_val, attention_mask_val = mask_tokens(x_true_val)
x_train[0]

array([ 0,  6,  7,  8,  9, 10,  7, 11, 12, 13, 14, 15, 16,  3, 18, 19, 20,
       21, 22, 19, 23, 24, 25,  1,  5,  5,  5,  5,  5,  5,  5,  5])

In [33]:
targets_train[:2]

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False]])

In [34]:
attention_mask[:2]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, False,
        False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False]])

In [35]:
x_true[:2]

array([[ 0,  6,  7,  8,  9, 10,  7, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 19, 23, 24, 25,  1,  5,  5,  5,  5,  5,  5,  5,  5],
       [ 0,  2, 14, 26, 27, 28,  7, 29, 30, 31, 32, 33,  7, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44,  6, 45, 46, 47, 48, 49,  1,  5]])

# Override model to include masking

In [36]:
from transformers import TFBertForMaskedLM, BertConfig

I0407 17:18:30.780511 46997179429888 file_utils.py:57] TensorFlow version 2.1.0 available.


## Overriding functions

In [37]:
class bert_with_mask(tf.keras.Model):
    def __init__(self, config, onehot_mask):
        super(bert_with_mask, self).__init__()
        self.bert = TFBertForMaskedLM(config)
        self.onehot_mask = onehot_mask

    def call(self, inputs):
        mask = inputs[-1] # unpack mask from inputs
        inputs = inputs[:-1]
        outputs = self.bert(inputs)[0]
        
        outputs = tf.where(mask[:,:,None], outputs, self.onehot_mask[None,None,:])
        
        return (outputs,)

## Define masking function

In [38]:
onehot_mask = np.zeros(len(my_tokenizer_lut), dtype=np.float32)
onehot_mask[my_tokenizer_lut["<PAD>"]] = 1.0

## Test the model

In [39]:
config = BertConfig.from_pretrained('../code/bert_config_tiny.json')
config.vocab_size = len(my_tokenizer_lut)

I0407 17:18:41.834565 46997179429888 configuration_utils.py:281] loading configuration file ../code/bert_config_tiny.json
I0407 17:18:41.837863 46997179429888 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 2,
  "num_beam

In [40]:
model2 = bert_with_mask(config, onehot_mask)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model2.compile(optimizer=optimizer, loss=loss)

# Define a dataset

In [44]:
BATCH_SIZE = 32
def data_generator_fn():
    for x in x_true:
        x_train, targets_train, masked_x_true, attention_mask = mask_tokens(x)
        yield (x_train, attention_mask, targets_train), masked_x_true

# dataset object
dataset = tf.data.Dataset.from_generator(
    data_generator_fn,
    output_types=((tf.int32, tf.bool, tf.bool), tf.int32),
    output_shapes=(( tf.TensorShape((SENTENCE_LENGTH,)), tf.TensorShape((SENTENCE_LENGTH,)), tf.TensorShape((SENTENCE_LENGTH,)) ), tf.TensorShape((SENTENCE_LENGTH,)) )
)
dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [41]:
BATCH_SIZE = 32
def data_generator_fn_val():
    for x in x_true_val:
        x_train, targets_train, masked_x_true, attention_mask = mask_tokens(x)
        yield (x_train, attention_mask, targets_train), masked_x_true

# dataset object
dataset_val = tf.data.Dataset.from_generator(
    data_generator_fn_val,
    output_types=((tf.int32, tf.bool, tf.bool), tf.int32),
    output_shapes=(( tf.TensorShape((SENTENCE_LENGTH,)), tf.TensorShape((SENTENCE_LENGTH,)), tf.TensorShape((SENTENCE_LENGTH,)) ), tf.TensorShape((SENTENCE_LENGTH,)) )
)
dataset_val = dataset_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
hist = model2.fit(dataset, validation_data=dataset_val, epochs=10)

Epoch 1/10


/localscratch/guest139.356254.0/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
W0407 17:21:43.190136 46997179429888 optimizer_v2.py:1043] Gradients do not exist for variables ['bert_with_mask/tf_bert_for_masked_lm/bert/pooler/dense/kernel:0', 'bert_with_mask/tf_bert_for_masked_lm/bert/pooler/dense/bias:0'] when minimizing the loss.
/localscratch/guest139.356254.0/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
W0407 17:21:45.591462 46997179429888 optimizer_v2.py:1043] Gradients do not exist for variables ['bert_with_mask/tf_ber

14813/14813 [==============================] - 1405s 95ms/step - loss: 9.2829 - val_loss: 9.2414
Epoch 2/10
 2064/14813 [===>..........................] - ETA: 20:01 - loss: 9.2382

In [ ]:
my_input = tokenize_string("I drove to <MASK> .")
my_output = model2((my_input, tf.ones(my_input.shape[:2], dtype=tf.bool) ))[0]

In [ ]:
tf.nn.top_k(my_output[0,4,:], 10)

In [ ]:
inv_tokenizer_lut = {v: k for k, v in my_tokenizer_lut.items()}
for t in range(7):
    print()
    print(f"t{t}:")
    for i in tf.nn.top_k(my_output[0,t,:], 5).indices.numpy():
        print(inv_tokenizer_lut[i])

# Save/Load model weights

In [ ]:
!rm -rf tinyBERT
!mkdir tinyBERT
model2.save_weights('tinyBERT/tinyBERT', save_format='tf')

In [ ]:
# Recreate the model
new_model = bert_with_mask(config, onehot_mask)
new_model.compile(optimizer=optimizer, loss=loss)

# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
new_model.train_on_batch(dataset.take(1))

# Load the state of the old model
new_model.load_weights('tinyBERT/tinyBERT')

In [ ]:
my_input = tokenize_string("i drove to work")
my_output = new_model((my_input, tf.ones(my_input.shape[:2], dtype=tf.bool) ))[0]

In [ ]:
for i in range(7):
    print(inv_tokenizer_lut[my_input[0,i]])

In [ ]:
for t in range(7):
    print()
    print(f"t{t}:")
    for i, pct in zip(tf.nn.top_k(my_output[0,t,:], 2).indices.numpy(), tf.nn.top_k(my_output[0,t,:], 2).values.numpy()):
        print(f"Guess: {inv_tokenizer_lut[i]}, pct: {pct}")